In [5]:
import pandas as pd
import geopy.distance
import numpy as np
import time
import math
from sklearn import preprocessing



In [49]:
crime_df.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,...,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
1,149013323,14.0,MANHATTAN,12/31/2018,23:40:00,12/31/2018,23:50:00,COMPLETED,NaN,NaN,...,NaN,NaN,45-64,WHITE,F,987866.0,214721.0,40.756044,-73.986950,"(40.75604353500006, -73.98694954399997)"
2,642981531,73.0,BROOKLYN,12/31/2018,23:30:00,12/31/2018,23:40:00,COMPLETED,NaN,NaN,...,F,NaN,18-24,WHITE HISPANIC,F,1006995.0,185258.0,40.675146,-73.918003,"(40.67514616200003, -73.91800323399997)"
3,429685363,67.0,BROOKLYN,12/31/2018,23:20:00,12/31/2018,23:30:00,COMPLETED,NaN,NaN,...,M,NaN,25-44,BLACK,F,999584.0,169903.0,40.633016,-73.944756,"(40.63301598000004, -73.94475559899996)"
4,290330841,5.0,MANHATTAN,12/31/2018,23:15:00,12/31/2018,23:20:00,COMPLETED,NaN,NaN,...,U,NaN,25-44,WHITE,M,986164.0,202140.0,40.721512,-73.993096,"(40.72151233500005, -73.99309646799998)"
5,304070477,46.0,BRONX,12/31/2018,23:15:00,12/31/2018,23:25:00,COMPLETED,NaN,NaN,...,F,NaN,25-44,BLACK,F,1009331.0,250025.0,40.852908,-73.909340,"(40.85290843100006, -73.90934005599996)"


In [215]:
listings_df = pd.read_csv("listings-detail.csv")

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (43,61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [216]:
listings_df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2539,https://www.airbnb.com/rooms/2539,20190306152833,2019-03-07,Clean & quiet apt home by the park,Renovated apt home in elevator building.,"Spacious, renovated, and clean apt home, one b...",Renovated apt home in elevator building. Spaci...,none,Close to Prospect Park and Historic Ditmas Park,...,f,f,moderate,f,f,6,0,5,1,0.23
1,2595,https://www.airbnb.com/rooms/2595,20190306152833,2019-03-07,Skylit Midtown Castle,"Find your romantic getaway to this beautiful, ...","- Spacious (500+ft²), immaculate and nicely fu...","Find your romantic getaway to this beautiful, ...",none,Centrally located in the heart of Manhattan ju...,...,f,f,strict_14_with_grace_period,t,t,2,1,0,1,0.38
2,3330,https://www.airbnb.com/rooms/3330,20190306152833,2019-03-07,++ Brooklyn Penthouse Guestroom ++,"This is a spacious, clean, furnished master be...","Room Features: - clean, hardwood floors - 2 la...","This is a spacious, clean, furnished master be...",none,Location is GREAT!! Right off the L train in t...,...,f,f,strict_14_with_grace_period,f,f,3,0,3,0,0.35
3,3647,https://www.airbnb.com/rooms/3647,20190306152833,2019-03-07,THE VILLAGE OF HARLEM....NEW YORK !,NaN,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,WELCOME TO OUR INTERNATIONAL URBAN COMMUNITY T...,none,NaN,...,f,f,strict_14_with_grace_period,t,t,1,0,1,0,NaN
4,3831,https://www.airbnb.com/rooms/3831,20190306152833,2019-03-07,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,...,f,f,moderate,f,f,1,1,0,0,4.43


In [206]:
crime_df = pd.read_csv("nyc_crime.csv")

In [207]:
updated_crime_df = crime_df[np.isfinite(crime_df['Latitude'])]

In [208]:
updated_crime_df = updated_crime_df[updated_crime_df['CMPLNT_FR_DT'] > '1/15/2019']

In [209]:
updated_crime_df.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,...,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
1,149013323,14.0,MANHATTAN,12/31/2018,23:40:00,12/31/2018,23:50:00,COMPLETED,NaN,NaN,...,NaN,NaN,45-64,WHITE,F,987866.0,214721.0,40.756044,-73.986950,"(40.75604353500006, -73.98694954399997)"
2,642981531,73.0,BROOKLYN,12/31/2018,23:30:00,12/31/2018,23:40:00,COMPLETED,NaN,NaN,...,F,NaN,18-24,WHITE HISPANIC,F,1006995.0,185258.0,40.675146,-73.918003,"(40.67514616200003, -73.91800323399997)"
3,429685363,67.0,BROOKLYN,12/31/2018,23:20:00,12/31/2018,23:30:00,COMPLETED,NaN,NaN,...,M,NaN,25-44,BLACK,F,999584.0,169903.0,40.633016,-73.944756,"(40.63301598000004, -73.94475559899996)"
4,290330841,5.0,MANHATTAN,12/31/2018,23:15:00,12/31/2018,23:20:00,COMPLETED,NaN,NaN,...,U,NaN,25-44,WHITE,M,986164.0,202140.0,40.721512,-73.993096,"(40.72151233500005, -73.99309646799998)"
5,304070477,46.0,BRONX,12/31/2018,23:15:00,12/31/2018,23:25:00,COMPLETED,NaN,NaN,...,F,NaN,25-44,BLACK,F,1009331.0,250025.0,40.852908,-73.909340,"(40.85290843100006, -73.90934005599996)"


In [210]:
def find_distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1))* math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    dist = radius * c

    return dist

In [211]:
def find_crime_score(listing_Coord, borrough_name):
    crime_score = 0    
    specific_crime_df = updated_crime_df[updated_crime_df['BORO_NM'] == borrough_name]
    
    for index, row in specific_crime_df.iterrows():
        crime_X = row['Latitude']
        crime_Y = row['Longitude']
        crime_Coord = (crime_X, crime_Y)
        if find_distance(listing_Coord, crime_Coord) < 0.5:
            crime_score +=1

    return crime_score

In [212]:
crime_score = []
for index, row in listings_df.iterrows():
    st = time.time()
    coord = (row['latitude'], row['longitude'])
    borrough_name = row['neighbourhood_group']
    borrough_name = borrough_name.upper()
    c = find_crime_score(coord, borrough_name)
    crime_score.append(c)    
        
listings_df['crime_score'] = crime_score

KeyboardInterrupt: 

In [195]:
listings_df.to_csv('listings_with_transitscore_and_crimescore.csv', sep='\t', encoding='utf-8')

In [22]:
listings_df_with_crime = pd.read_csv('listings_with_transitscore_and_crimescore.csv', sep='\t', encoding='utf-8')


In [23]:
listings_df_with_crime.describe()

,Unnamed: 0,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,transitscore,crime_score
count,49748.000000,4.974800e+04,4.974800e+04,49748.000000,49748.000000,49748.000000,49748.000000,49748.000000,39260.000000,49748.000000,49748.000000,49748.000000,49748.000000
mean,24873.500000,1.738967e+07,6.058319e+07,40.729299,-73.953238,150.931796,7.064284,22.244894,1.300503,5.742261,118.296193,4.640910,340.601331
std,14361.154933,9.816381e+06,6.951425e+07,0.053860,0.044968,225.025933,20.481797,41.954729,1.563485,20.767163,134.379128,0.854305,208.530474
min,0.000000,2.539000e+03,2.571000e+03,40.499790,-74.242850,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000,0.000000,0.000000
25%,12436.750000,9.004913e+06,7.426523e+06,40.690750,-73.983500,69.000000,1.000000,1.000000,0.207500,1.000000,0.000000,5.000000,184.000000
50%,24873.500000,1.834661e+07,2.899476e+07,40.723670,-73.956450,107.000000,2.000000,5.000000,0.670000,1.000000,61.000000,5.000000,298.000000
75%,37310.250000,2.609943e+07,9.698084e+07,40.763130,-73.937530,175.000000,5.000000,23.000000,1.910000,2.000000,239.000000,5.000000,460.000000
max,49747.000000,3.285890e+07,2.471184e+08,40.911690,-73.712990,10000.000000,1250.000000,588.000000,19.360000,192.000000,365.000000,5.000000,1797.000000


In [24]:
def normalize(arr):
    result = arr.copy()
    for i in range(0,len(result)):
        max_value = result.max()
        min_value = result.min()
        result[i] = (result[i] - min_value) / (max_value - min_value)
    return result

# min_max_scaler = preprocessing.MinMaxScaler()
np_arr = listings_df_with_crime['crime_score']
newnp_arr = np.array(np_arr).astype('float64')

print ("array before normalizing - {}".format(np_arr))
newnp_arr = normalize(newnp_arr)

print ("array after normalizing - {}".format(newnp_arr))
listings_df_with_crime['crime_score_normalized'] = newnp_arr
# np_scaled = min_max_scaler.fit_transform(np_val)

#listings_df_with_crime['crime_score_normalized'] = listings_df_with_crime['crime_score'].apply(lambda x: x/x.max())


array before normalizing - 0         88
1        676
2        310
3        641
4        248
5        724
6        329
7        260
8        773
9        294
10       620
11       446
12       264
13       221
14       924
15       158
16       268
17       374
18       322
19       560
20        70
21       206
22       259
23       255
24       247
25       635
26       443
27       389
28       410
29       275
        ... 
49718     79
49719    307
49720    310
49721     63
49722     68
49723    109
49724    517
49725    612
49726    280
49727    257
49728    101
49729    772
49730    573
49731    322
49732     23
49733    112
49734    298
49735    298
49736    331
49737    256
49738     90
49739    376
49740    479
49741    860
49742    316
49743    182
49744    318
49745     22
49746    294
49747    377
Name: crime_score, Length: 49748, dtype: int64
array after normalizing - [0.04897051 0.37618253 0.17250974 ... 0.05835544 0.77984085 1.        ]


In [26]:
listings_df_with_crime['crime_score_normalized'].describe()

listings_df_with_crime.to_csv("listings_with_transitscore_and_crimescore_normalized.csv")

In [27]:
listings_df_with_crime

,Unnamed: 0,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,transitscore,crime_score,crime_score_normalized
0,0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.23,6,365,5,88,0.048971
1,1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,43,2019-01-02,0.38,2,365,5,676,0.376183
2,2,3330,++ Brooklyn Penthouse Guestroom ++,4177,Jbee,Brooklyn,Williamsburg,40.70856,-73.94236,Private room,70,5,39,2018-12-07,0.35,3,217,5,310,0.172510
3,3,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365,5,641,0.356706
4,4,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,239,2019-03-03,4.43,1,164,5,248,0.138008
5,5,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.11,1,12,5,724,0.402894
6,6,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,185,2,70,2018-12-02,0.58,1,18,5,329,0.183083
7,7,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,45,49,2017-10-05,0.41,1,0,5,260,0.144686
8,8,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,415,2019-02-17,3.47,1,158,5,773,0.430161
9,9,5203,Cozy Clean Guest Room - Family Apt,7490,MaryEllen,Manhattan,Upper West Side,40.80178,-73.96723,Private room,79,2,118,2017-07-21,1.02,1,0,5,294,0.163606
